In [1]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedKFold

# AMFD_Dataframe = pd.read_excel('AMFD_Norming_Data_Table.xlsx', skiprows=4)

In [2]:
#########################################Stracth Part############################################
# import torch
# from torchvision.transforms import Compose, Resize, ToTensor
# from PIL import Image
# import os

# # image paths
# image_dir = "AMFD_Faces_Final"
# image_paths = []
# for filename in os.listdir(image_dir):
#     if filename.endswith('.jpg'):
#         image_paths.append(os.path.join(image_dir, filename))

# # photoid = 1001+idx
# idx = 4

# # print idx-th image
# image_path = image_paths[idx]
# print(image_path)

# # Features tensor get
# features = torch.tensor(AMFD_Dataframe.iloc[idx, 0:].values, dtype=torch.float32)
# print(features)
# print(len(features))

# # Image ID get
# image_id = AMFD_Dataframe.iloc[idx, 0]
# print(image_id)

# # Expression and Gender get as label
# #image_path = "AMFD_Faces_Final\SF-1040.jpg"
# parts = image_path.split('\\')[-1].split('-')
# expression_code = parts[0][0]
# gender_code = parts[0][1]
# expression_label = 0 if expression_code == 'N' else 1
# gender_label = 0 if gender_code == 'F' else 1
# print(f"{image_path}'s expression code is {expression_code} and gender code is {gender_code}")

# # image to tensor
# image = Image.open(image_path).convert('RGB')
# image_tensor = ToTensor()(image)
# print(image_tensor)

# # check if image is all white or not
# any_value_over_1 = (image_tensor == 1).all().item()
# max_value = image_tensor.max().item()
# any_value_over_1, max_value

#########################################Stracth Part############################################

In [3]:
import pandas as pd
# Read excel table and set data to data frame
AMFD_Dataframe = pd.read_excel('AMFD_Norming_Data_Table.xlsx', skiprows=4)

In [4]:
import os
# put all image path to a list
image_dir = "AMFD_Faces_Final"
image_paths = []
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        image_paths.append(os.path.join(image_dir, filename))
        

In [5]:
import torch
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import Dataset

class AMFDDataset(Dataset):
    def __init__(self, dataframe, image_paths, transform=None):
        self.dataframe = dataframe
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        
        image_path = self.image_paths[idx]

        # extract element from the image name
        parts = image_path.split('\\')[-1].split('-')

        # get expression code (which is the Ftype in table)
        expression_code = 0 if parts[0][0] == 'N' else 1

        # get image id from image file
        image_id_photo = parts[1].split('.')[0]

        # using expression code and image id to seach in dataframe
        match = self.dataframe[(self.dataframe['PhotoID'] == int(image_id_photo)) & (self.dataframe['FType'] == expression_code)]
        
        # get the feature from the matching line
        image_features = None
        if not match.empty:
            # this will return the whole line 
            # if we need photo_id included, use match.iloc[0, 0:].values, otherwise match.iloc[0, 1:].values
            image_features = torch.tensor(match.iloc[0, 0:].values, dtype=torch.float32)
        else:
            raise ValueError("No data found")

        # convert image to tensor. if transform needed, using transform
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        else:
            image = ToTensor()(image)

        return image_id_photo, image, image_features 
    
    # (X - mean) / sd normalization
    def normalize(self, df):
        normalized_df = df.copy()
        # first column is photo id, skip
        for column in df.columns[1:]: 
            mean_value = df[column].mean()
            std_value = df[column].std()
            normalized_df[column] = (df[column]-mean_value)/std_value
        return normalized_df
    

c:\Study\Softwares\Anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Study\Softwares\Anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
import numpy as np

# Create data set from data frame
amfd_dataset = AMFDDataset(AMFD_Dataframe, image_paths)

# using photo ids as unit because we must put photos of one person in one dataset
photo_ids = AMFD_Dataframe['PhotoID'].values
unique_photo_ids = np.unique(photo_ids)

# extract a single set (1/10) for test set
train_val_photo_ids, test_photo_ids = train_test_split(unique_photo_ids, test_size=0.1, random_state=42)

# extract a single set for validation set.
train_photo_ids, val_photo_ids = train_test_split(train_val_photo_ids, test_size=(1/9), random_state=42) 

In [7]:
from torch.utils.data import DataLoader

# get index from train_photo_ids, val_photo_ids, and test_photo_ids
def get_photo_ids(photo_id_sets):
    result = []
    for i, photo_id in enumerate(photo_ids):
        if photo_id in photo_id_sets:
            result.append(i)
    return result

train_indices = get_photo_ids(train_photo_ids)
val_indices = get_photo_ids(val_photo_ids)
test_indices = get_photo_ids(test_photo_ids)

print(f"train_indices is {train_indices} and len is {len(train_indices)}")
print(f"val_indices is {val_indices} and len is {len(val_indices)}")
print(f"test_indices is {test_indices} and len is {len(test_indices)}")

# using these indices to get element from dataset in order to make photos of one person in one set
train_dataset = Subset(amfd_dataset, train_indices)
val_dataset = Subset(amfd_dataset, val_indices)
test_dataset = Subset(amfd_dataset, test_indices)

print(f"train_dataset is {train_dataset} and len is {len(train_dataset)}")
print(f"val_dataset is {val_dataset} and len is {len(val_dataset)}")
print(f"test_dataset is {train_dataset} and len is {len(test_dataset)}")

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


train_indices is [0, 2, 3, 5, 6, 7, 8, 9, 12, 13, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29, 31, 32, 33, 34, 37, 38, 39, 40, 41, 42, 43, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 81, 82, 83, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 114, 115, 116, 117, 118, 121, 122, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 137, 138, 140, 141, 142, 143, 146, 147, 148, 149, 150, 151, 152, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 171, 172, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 190, 191, 192, 194, 195, 196, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217] and len is 174
val_indices is [1, 14, 21, 27, 35, 36, 44, 46, 61, 80, 88, 110, 123, 130, 136, 144, 145, 153, 155, 170, 189, 197] and len is 22
test_indices is [4, 10, 11, 30, 45, 64, 68, 78, 79, 84,